# Problema de transporte discreto

Dadas dos distribuciones de probabilidad discretas:

* $X$: con soporte $x_1<x_2<\ldots<x_m$ con probabilidades $P(X=x_i) = p_i$.
* $Y$: con soporte $y_1<y_2<\ldots<y_n$ con probabilidades $P(Y=y_j) = q_j$.

Queremos hallar una distribución conjunta $A=(a_{ij}) \geqslant 0$ tal que:

$$\min_{a_{ij}} \sum_{i,j} a_{ij} |x_i-y_j|$$

sujeto a:

$$\sum_j a_{ij} = p_i \quad \forall i=1,\ldots,m,$$
$$\sum_i a_{ij} = q_j \quad \forall j=1,\ldots,n.$$

In [1]:
## Para instalar los paquetes del Project.toml la primera vez
#using Pkg; Pkg.instantiate()

In [2]:
using JuMP, GLPK, Ipopt

#m=3
x=[1.0;2.0;3.0]
p=[0.2;0.4;0.4]

#n=2
y=[0;4.0]
q=[0.5;0.5]

model = JuMP.Model(GLPK.Optimizer)
#model = JuMP.Model(Ipopt.Optimizer)


m=length(x)
n=length(y)

#Calculo los pesos |xi-yj|
W=abs.(x*ones(1,n) - ones(m,1)*y')

@variable(model,A[1:m,1:n]>=0)

conp = @constraint(model, sum(A, dims=2).==p)
conq = @constraint(model, sum(A, dims=1).==q')

@objective(model,Min, sum( A.*W ))

model

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: A

In [3]:
optimize!(model)

In [4]:
objective_value(model)

1.4

In [5]:
#Extraigo el valor optimo
value.(A)

3×2 Matrix{Float64}:
 0.2  0.0
 0.3  0.1
 0.0  0.4

In [6]:
#Extraigo los multiplicadores de p
lambda = dual.(conp)

3×1 Matrix{Float64}:
 1.0
 2.0
 1.0

In [7]:
#Extraigo los multiplicadores de q
mu = dual.(conq)

1×2 Matrix{Float64}:
 0.0  0.0

In [8]:
W

3×2 Matrix{Float64}:
 1.0  3.0
 2.0  2.0
 3.0  1.0

In [9]:
lambda * ones(1,n) + ones(m,1)*mu

3×2 Matrix{Float64}:
 1.0  1.0
 2.0  2.0
 1.0  1.0